Newton Raphson

In [35]:
import numpy as np
import pandas as pd


B_mat = np.matrix([[-18, 10, 8], [10, -22, 12], [8, 12, -20]])


# converts actual index to zero-based
def B(row, col):
    return B_mat[row - 1, col - 1]


# known variables
V_1 = 1.02
V_2 = 1.01
V_3 = 1.0
P_2 = 0.7
P_3 = -2


def jacobian_inverse(x):
    delta_2 = x[0]
    delta_3 = x[1]

    eq1 = V_2 * (
        V_1 * B(2, 1) * np.cos(delta_2) + V_3 * B(2, 3) * np.cos(delta_2 - delta_3)
    )
    eq2 = -V_2 * V_3 * B(2, 3) * np.cos(delta_2 - delta_3)
    eq3 = -V_2 * V_3 * B(3, 2) * np.cos(delta_3 - delta_2)
    eq4 = V_3 * (
        V_1 * B(3, 1) * np.cos(delta_3) + V_2 * B(3, 2) * np.cos(delta_3 - delta_2)
    )

    J = np.matrix(
        [
            [
                eq1,
                eq2,
            ],
            [
                eq3,
                eq4,
            ],
        ]
    )

    return np.linalg.inv(J)


# flat start
# [[delta2, delta3],...]
deltas = [np.array([0, 0])]

POWER_TOLERANCE = 10e-3

i = 0
while True:
    i += 1
    d2 = deltas[-1][0]
    d3 = deltas[-1][1]

    mismatch_P2 = P_2 - (
        V_2 * (V_1 * B(2, 1) * np.sin(d2) + V_3 * B(2, 3) * np.sin(d2 - d3))
    )
    mismatch_P3 = P_3 - (
        V_3 * (V_1 * B(3, 1) * np.sin(d3) + V_2 * B(3, 2) * np.sin(d3 - d2))
    )
    mismatch_P = np.matrix([[mismatch_P2], [mismatch_P3]])

    d = deltas[-1].reshape(2, 1)
    new_deltas = np.add(
        d,
        jacobian_inverse(deltas[-1]).dot(mismatch_P),
    )

    deltas.append(np.ravel(new_deltas))

    # convergence condition
    if np.abs(mismatch_P2) < POWER_TOLERANCE and np.abs(mismatch_P3) < POWER_TOLERANCE:
        break


print(f"took {i} iterations to converge")
df = pd.DataFrame(
    {
        "iteration": np.arange(1, i + 1),
        "angle for bus 2 [radians]": [xx[0] for xx in deltas[1:]],
        "angle for bus 3 [radians]": [xx[1] for xx in deltas[1:]],
    }
)


d2 = deltas[-1][0]
d3 = deltas[-1][1]
Q_sync_condenser = 0.5 + -V_3 * (
    V_1 * B(1, 3) * np.cos(d3) + V_2 * B(2, 3) * np.cos(d3 - d2) + V_3 * B(3, 3)
)

print("Q_sync_condenser:", Q_sync_condenser)

df

took 2 iterations to converge
Q_sync_condenser: 0.321478375938554


,iteration,angle for bus 2 [radians],angle for bus 3 [radians]
0,1,-0.032629,-0.118120
1,2,-0.032688,-0.118328


Fast Decoupled

In [37]:
import numpy as np
import pandas as pd


B_mat = np.matrix([[-18, 10, 8], [10, -22, 12], [8, 12, -20]])
B_P = np.matrix([[22, -12], [-12, 20]])
B_P_inv = np.linalg.inv(B_P)


# converts actual index to zero-based
def B(row, col):
    return B_mat[row - 1, col - 1]


# known variables
V_1 = 1.02
V_2 = 1.01
V_3 = 1.0
P_2 = 0.7
P_3 = -2

# flat start
# [[delta2, delta3],...]
deltas = [np.array([0, 0])]

POWER_TOLERANCE = 10e-3

i = 0
while True:
    i += 1
    d2 = deltas[-1][0]
    d3 = deltas[-1][1]

    matrix_A = B_P_inv.dot(np.linalg.inv(np.diag([V_2, V_3])))

    mismatch_P2 = P_2 - (
        V_2 * (V_1 * B(2, 1) * np.sin(d2) + V_3 * B(2, 3) * np.sin(d2 - d3))
    )
    mismatch_P3 = P_3 - (
        V_3 * (V_1 * B(3, 1) * np.sin(d3) + V_2 * B(3, 2) * np.sin(d3 - d2))
    )
    mismatch_P = np.matrix([[mismatch_P2], [mismatch_P3]])

    d = deltas[-1].reshape(2, 1)
    new_deltas = np.add(d, matrix_A.dot(mismatch_P))

    deltas.append(np.ravel(new_deltas))

    # convergence condition
    if np.abs(mismatch_P2) < POWER_TOLERANCE and np.abs(mismatch_P3) < POWER_TOLERANCE:
        break


print(f"took {i} iterations to converge")
df = pd.DataFrame(
    {
        "iteration": np.arange(1, i + 1),
        "angle for bus 2 [radians]": [xx[0] for xx in deltas[1:]],
        "angle for bus 3 [radians]": [xx[1] for xx in deltas[1:]],
    }
)


d2 = deltas[-1][0]
d3 = deltas[-1][1]
Q_sync_condenser = 0.5 + -V_3 * (
    V_1 * B(1, 3) * np.cos(d3) + V_2 * B(2, 3) * np.cos(d3 - d2) + V_3 * B(3, 3)
)

print("Q_sync_condenser:", Q_sync_condenser)

df

took 3 iterations to converge
Q_sync_condenser: 0.3214788277576588


,iteration,angle for bus 2 [radians],angle for bus 3 [radians]
0,1,-0.034252,-0.120551
1,2,-0.032654,-0.118294
2,3,-0.032689,-0.118329
